In [1]:

import matplotlib
import pandas as pd
import numpy as np

import pickle 
import os
import re
import inspect
from importlib import reload as irl

from collections import defaultdict
from itertools import  chain
from dataclasses import dataclass


print(pd.__version__)

VERBOSE = False

0.24.2


##### Load Nominations

ToDo
1. more than 1000 articles in duplicates?

Visualisierungen
1. Multinominierung


##### Load ends, transform it to a DataFrame and keep a set of all article names

# I. Summarize and clean data

In [2]:
# Load results of parsing monthly archives
df_FAC = pd.read_csv('./data/FAC_nomination.csv', sep=';', index_col=0, parse_dates=['nomination', 'last_comment'])  
df_FAC = df_FAC[(np.datetime64('2005')<=df_FAC.nomination) & (df_FAC.nomination<=np.datetime64('2016'))]

# Load results of parsing revision history 
with open('./data/FAC_ends.pkl', 'rb') as file:
    ends_dict = pickle.load(file)
    
# convert revision histotory from dictionary to pd.DataFrame
df_ends = [[i, article, dates]for i, (article, dates) in enumerate(ends_dict.items())]
df_ends = pd.DataFrame(df_ends, columns=['idx', 'title', 'dates'])

# pd.DataFrame with min/max date
df_ends_min = [[i, article, max(dates), min(dates)] for i, (article, dates) in enumerate(ends_dict.items())]
df_ends_min = pd.DataFrame(df_ends_min, columns=['idx', 'title', 'first', 'last'])

# clean article names (min-version)
# it's possible to increase  number of matching articles, by preprocessing
df_FAC['title'] = df_FAC.title.str.replace('/archive\d', '') # These are actually unique nominations
df_ends['title'] = df_ends.title.str.replace('/archive\d', '') # These likely not
df_ends_min['title'] = df_ends_min.title.str.replace('/archive\d', '')

# This will produce duplicates,
#   1) append all dates
df_ends = df_ends.groupby('title').agg({'dates' :'sum'})
#df_ends.reset_index(inplace=True)

##  2) chose min, max
agg_funcs = {'first': 'min', 
             'last' : 'max'}
df_ends_min = df_ends_min.groupby('title').agg(agg_funcs)


n_FAC = len(df_FAC)
n_ends = len(df_ends)

# We only keep the first nomination (simplification)
df_FAC['has_duplicate'] = df_FAC.duplicated(subset='title', keep=False)
print('duplicates:')
print(f'multiple nominations: {sum(df_FAC.has_duplicate)}')
df_FAC = df_FAC.loc[~df_FAC.sort_values('nomination').duplicated(subset='title', keep='first'),]

# merge
df_merge = pd.merge(df_FAC, df_ends, on='title')
n_FAC_nd = len(df_FAC)
print(f'number of nominations: {n_FAC:>20} \nartilcles in ends: {n_ends:>20}'
      f'\nunique articles in nominations: {n_FAC_nd:10} '
      f'\narticles in merged data Frame: {len(df_merge):15}')

df_merge.to_csv('./tmp/FAC_merge.csv', sep=';')

duplicates:
multiple nominations: 1845
number of nominations:                 4966 
artilcles in ends:                10451
unique articles in nominations:       3885 
articles in merged data Frame:            3774


# II. Decide End
(Didn't manage to do this pandas so I went back to dictionaries)

In [5]:
with open('./data/FAC_ends.pkl', 'rb') as file:
    ends_dict = pickle.load(file)

res = defaultdict(list)
for article, dates in ends_dict.items():
    article = re.sub('/archive\d', '', article)
    dates = np.array([np.datetime64(date) for date in dates])
    res[article].append(dates)
ends_dict = res

# get set of titles
art_set = set(ends_dict.keys())

# read df_FAC (post I.)
print(len(df_merge))
df_merge = df_merge.loc[df_merge.title.isin(art_set),]
print(len(df_merge))

# convert pd.DataFrame to dict
FAC_dict = {k: np.datetime64(v) for k,v in df_merge[['title', 'last_comment']].values}
print(len(FAC_dict), len(ends_dict))

new_dict = {}
count =0
for article, last_date in FAC_dict.items():
    two_weeks_later = last_date + np.timedelta64(2, 'W')
    dates = list(chain.from_iterable(ends_dict[article]))   # just to be sure list is falttened
    dates = [date for date in dates if date < two_weeks_later]
    # print(two_weeks_later, type(dates), type(dates[0]), type(two_weeks_later))
    #new_date = np.max(dates) if dates else last_date        
    if dates:
        new_date  = max(dates)
    else:
        new_date = last_date
        print(f'No date (< 2W) found for article: {article}')
    new_dict[article] = new_date
    
df_new = pd.DataFrame({'title' : list(new_dict.keys()), 'end_date' : list(new_dict.values())})
df_merge = pd.merge(df_merge, df_new, on='title')

df_merge.head()


3774
3774
3774 10451
No date (< 2W) found for article: Adventure Time
No date (< 2W) found for article: Taylor Swift
No date (< 2W) found for article: Jack Parsons (rocket engineer)
No date (< 2W) found for article: Potential cultural impact of extraterrestrial contact
No date (< 2W) found for article: Fluorine
No date (< 2W) found for article: Ashley Tisdale
No date (< 2W) found for article: Borat: Cultural Learnings of America for Make Benefit Glorious Nation of Kazakhstan
No date (< 2W) found for article: iPod
No date (< 2W) found for article: Pixies (band)
No date (< 2W) found for article: West Bengal
No date (< 2W) found for article: Paul Martin
No date (< 2W) found for article: Stanley Kubrick
No date (< 2W) found for article: Moncton
No date (< 2W) found for article: Freemasonry
No date (< 2W) found for article: Stonewall riots
No date (< 2W) found for article: Earth
No date (< 2W) found for article: Funeral of Pope John Paul II
No date (< 2W) found for article: Mars
No date (< 

,idx,title,nomination,last_comment,has_duplicate,dates,end_date_x,end_date_y
0,434,Belgium national football team,2015-11-21 10:28:00,2016-02-05 08:07:00,False,"[2015-11-21 10:32:10, 2015-11-21 12:36:21, 201...",2016-02-05 08:56:59,2016-02-05 08:56:59
1,435,Bicycle kick,2015-12-27 04:23:00,2016-02-13 15:59:00,False,"[2015-12-27 04:24:35, 2015-12-27 14:09:14, 201...",2016-02-13 14:47:32,2016-02-13 14:47:32
2,443,Mullum Malarum,2015-11-09 07:51:00,2016-02-06 19:41:00,False,"[2015-11-09 07:49:01, 2015-11-09 09:00:07, 201...",2016-02-05 23:08:09,2016-02-05 23:08:09
3,449,2012 Gatorade Duels,2015-12-21 15:14:00,2016-01-29 22:10:00,False,"[2015-12-21 15:15:45, 2015-12-23 15:58:09, 201...",2016-01-29 22:03:31,2016-01-29 22:03:31
4,450,Briarcliff Farms,2015-12-25 00:20:00,2016-01-29 22:22:00,False,"[2015-12-25 00:25:15, 2015-12-25 23:08:08, 201...",2016-02-11 01:11:48,2016-02-11 01:11:48


In [4]:
i = 0
for article, dates in art_dict.items():
    dates = np.array([np.datetime64(date) for date in dates])
    print(f'min:{min(dates)} max:{max(dates)}')
    p1 = dates[(dates- dates[0]) > np.timedelta64(30, 'D')]
    p2 = dates[~((dates- dates[0]) > np.timedelta64(30, 'D'))]
    print(article)
    print(len(p1), len(p2))
    if i >= 1:
        break
    i+=1

from pprint import pprint
pprint(res['Belgium national football team'])

NameError: name 'art_dict' is not defined

In [ ]:
df_FAC = pd.read_csv('./data/FAC_nomination.csv', sep=';', index_col=0, parse_dates=['nomination', 'last_comment'])  
d = df_FAC.groupby('title').count().sort_values('idx')
len(d.loc[d.idx >= 4])

In [ ]:
with open('./res/article_dict.pkl', 'rb') as file:
    ends_dict = pickle.load(file)
    
df_ends = [[i, article, set(dates)]for i, (article, dates) in enumerate(ends_dict.items())]
df_ends = pd.DataFrame(df_ends)

df_ends

